In [1]:
import torch
import torch.nn as nn

In [2]:
from bnelearn.strategy import NeuralNetStrategy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import FirstPriceSealedBidAuction

In [3]:
from copy import deepcopy

In [4]:
hidden_layer = 10
batch_size = 10
u_lo = 0
u_hi = 10
n_players = 2
n_items = 1

epochs = 100

In [5]:
model = NeuralNetStrategy(2, hidden_layer).cuda()

In [6]:
mechanism = FirstPriceSealedBidAuction(cuda=True)

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

In [8]:
p1 =Bidder.uniform(u_lo,u_hi, model, batch_size = batch_size, n_players=2)

In [9]:
p2 = Bidder.uniform(u_lo,u_hi, model, batch_size = batch_size, n_players=2)

In [10]:
p1.draw_valuations_(), p2.draw_valuations_();

In [11]:
b1 = p1.get_action()
b2 = p2.get_action()

In [12]:
for epoch in range(epochs):
    # draw new valuations and bids
    p1.draw_valuations_(), p2.draw_valuations_()    
    b1 = p1.get_action()
    b2 = p2.get_action()    
    bids = torch.cat((b1,b2), 1).view(batch_size, n_players, 1)
    
    # run the auction and calculate utilities
    alloc, pay = mechanism.run(bids)
    
    u1 = p1.get_utility(alloc[:,0,:], pay[:,0]).mean()
    u2 = p2.get_utility(alloc[:,1,:], pay[:,1]).mean()

    loss = -u2
    
    if epoch % 50 == 0:
        print('epoch ', epoch, 'avg bid', (b1/p1.valuations).mean().item(), (b2/p2.valuations).mean().item(), 'utility', u1.item(), u2.item() )
    
    # propagate loss and update parameters   
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

epoch  0 avg bid 0.0050779846496880054 0.0 utility 4.884816646575928 0.0
epoch  50 avg bid 0.0 0.0 utility 5.5091118812561035 0.0


# Same with ES

In [13]:
from collections import Iterable, deque
import torch
from torch.optim.optimizer import Optimizer, required
from copy import deepcopy

In [14]:
import torch.nn.utils as ut

In [15]:
from bnelearn.optimizer import ES
from bnelearn.environment import Environment

In [16]:
u_lo =0
u_hi =10

In [17]:
def strat_to_bidder(strategy, batch_size):
    return Bidder.uniform(u_lo,u_hi, strategy, batch_size = batch_size, n_players=2)

In [18]:
from bnelearn.strategy import TruthfulStrategy

In [19]:
model = NeuralNetStrategy(2, hidden_layer).cuda()
#model = TruthfulStrategy(2).cuda()

In [20]:
mechanism = FirstPriceSealedBidAuction(cuda=True)

In [21]:
batch_size = 1024

In [22]:
env = Environment(mechanism, [], max_env_size=1, batch_size=batch_size,
                  n_players=2, strategy_to_bidder_closure=strat_to_bidder)

In [23]:
es = ES(model=model, environment=env, lr=3e-3, sigma=.01, n_perturbations= 64)

In [24]:
epoch=1001

In [ ]:
for e in range(epoch):
    if e%25==0:
        print("epoch {}".format(e))
        bidder = strat_to_bidder(model, batch_size)
        bidder.draw_valuations_()
        share = (bidder.get_action() / bidder.valuations).mean()
        print("\tbid: {:2f}".format(share.item()))
        print("\tutility: {:2f}".format(-es.step().item()))
    else: es.step()
    

epoch 0
	bid: 0.026136
	utility: 3.314161
epoch 25
	bid: 0.041353
	utility: 3.075154
epoch 50
	bid: 0.058430
	utility: 3.311540
